2.3
multinomial naive Bayes model (with no smoothing) for SPAM filtering¶

Systems test your code first with the Chinese Example and show the resulting model.

Learn a SPAM filtering model from the ENRON dataset provided above. Save the model to file SPAM_Model_MNB.tsv.

Show the top 10 terms alphabetically sortig the words increasing and their corresponding model entries.
Write a mapreduce job to accomplish this. 
Show the bottom 10 terms also.

In [1]:
%%writefile ChineseExampleTrain.txt
D1	1	Chinese Beijing Chinese
D2	1	Chinese Chinese Shanghai
D3	1	Chinese Macao
D4	0	Tokyo Japan Chines

Overwriting ChineseExampleTrain.txt


In [18]:
%%writefile ChineseExampleTest.txt
D1	1	Chinese Beijing Chinese
D2	1	Chinese Chinese Shanghai
D3	1	Chinese Macao
D4	0	Tokyo Japan Chinese
D5	0	Chinese Chinese Chinese Tokyo

Overwriting ChineseExampleTest.txt



In [5]:
%%writefile mapper322.py
#!/usr/bin/env python

import sys
import re
from collections import defaultdict

for line in sys.stdin:
    try:
        iD, spam, subject, body = line.split('\t')
        body = body + ' ' + subject
    except ValueError:
        iD, spam, body = line.split('\t')
    
    wordsInThisEmail = defaultdict(int)    
    for word in re.findall(r"[\w']+", body):
        word = word.lower()
        wordsInThisEmail[word] += 1
    for word in wordsInThisEmail:
        #need id to mark new email -if checking frequncy in spam emails but probably don't need this
        #or probably need to count the number of spam and ham emails
        print iD , "\t" , spam, "\t", word , "\t" , wordsInThisEmail[word]

Writing mapper322.py


In [6]:
%%writefile reducer322.py
#!/usr/bin/env python

import sys
import re

from collections import defaultdict
spamWords = defaultdict(int)
hamWords = defaultdict(int)
spamCount = 0
hamCount = 0
vocab = set() 

p_word_spam = defaultdict(int)
p_word_ham = defaultdict(int)

for line in sys.stdin:
    iD, spam, word, count = line.split('\t')
    vocab.add(word)
    if(int(spam) == 1):
        spamWords[word] += int(count)
        spamCount += 1
    else:
        hamWords[word] += int(count)
        hamCount +=1


for word in vocab:
    #will give 0 for viagara if it was never there in ham
    #print "k"
    #if(spamCount != 0 & hamCount != 0): #just in case, you never know4
        #print "k"
    p_word_spam[word] = (spamWords[word]*1.00/spamCount*1.0)
    p_word_ham[word] = hamWords[word]*1.0000/hamCount*1.0

    print word, "\t", p_word_ham[word], "\t", p_word_spam[word] #, "\t", p_word_ham[word]


Writing reducer322.py


In [8]:
!chmod a+x mapper322.py
!chmod a+x reducer322.py

In [9]:
!echo "D4	0	Tokyo Japan Chinese" | ./mapper321.py

D4 	0 	japan 	1
D4 	0 	chinese 	1
D4 	0 	tokyo 	1


In [12]:
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -mapper /home/cloudera/Downloads/scalingML-master/mapper322.py \
   -reducer /home/cloudera/Downloads/scalingML-master/reducer322.py \
   -input ch.txt\
   -output chineseExmodel

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob1190655386064476820.jar tmpDir=null
16/11/28 07:58:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/11/28 07:58:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/11/28 07:58:39 INFO mapred.FileInputFormat: Total input paths to process : 1
16/11/28 07:58:39 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1281)
	at java.lang.Thread.join(Thread.java:1355)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:862)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:600)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:789)
16/11/28 07:58:39 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Meth

In [14]:
!hdfs dfs -cat chineseExmodel/part-0000*
!hdfs dfs -get chineseExmodel/part-00000 cp2.txt

shanghai  	0.0 	0.166666666667
tokyo  	0.333333333333 	0.0
chinese  	0.333333333333 	0.833333333333
macao  	0.0 	0.166666666667
japan  	0.333333333333 	0.0
beijing  	0.0 	0.166666666667


In [20]:
#!hadoop fs -get /home/cloudera/chmodelFinal/part-00000 chModel.txt
#!ls
import io
from collections import defaultdict

f = open('cp2.txt', "r")

## use readlines to read all lines in the file
## The variable "lines" is a list containing all lines
lines = file.read(f).splitlines()
#print lines

modelSpam = {}
modelHam = {} #defaultdict(int)

for line in lines:
        #print line
        word, h, s = line.split('\t')
        word = word.strip(" ")
        modelHam[word] = float(h) *1.0
        modelSpam[word] = float(s) *1.0
##print modelSpam
#print float(modelHam['chinese'])
f.close()

#get the lines from mapper
currentID = 0
vocab = set() # all the words in email
accurate = 0
total = 0
f = open('/home/cloudera/Downloads/scalingML-master/ChineseExampleTest.txt', "r")


## use readlines to read all lines in the file
## The variable "lines" is a list containing all lines
lines = file.read(f).splitlines()
#a="shanghai shangha 6 7"


#print a.split('\t')
for line in lines:
    
    #print line
    iD, spam, words = line.split('\t')
    p_h = 1.0
    p_s = 1.0
    for word in words.split():
        word = word.lower()
        p_h  = p_h*float(modelHam[word])
        p_s =  p_s*float(modelSpam[word])

    if(int(spam)== 1):
        c = "SPAM"
    else:
        c = "HAM"

    if p_s > p_h:
        p_c = "SPAM"
    else:
        p_c = "HAM"

    print words, "\t ", p_c, " \t ", c

    if(p_c == c):

        accurate += 1
    total +=1

    # vocab.add(word)
    #currentID = iD
        
print "Accuracy: ", (accurate*1.0/total)

Chinese Beijing Chinese 	  SPAM  	  SPAM
Chinese Chinese Shanghai 	  SPAM  	  SPAM
Chinese Macao 	  SPAM  	  SPAM
Tokyo Japan Chinese 	  HAM  	  HAM
Chinese Chinese Chinese Tokyo 	  HAM  	  HAM
Accuracy:  1.0


In [23]:
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -D mapreduce.partition.keycomparator.options="-k1,1" \
   -mapper /home/cloudera/Downloads/scalingML-master/mapper322.py \
   -reducer /home/cloudera/Downloads/scalingML-master/reducer322.py \
   -input NB/e3.txt\
   -output nbmodelNoSmooth

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob1924883266190749431.jar tmpDir=null
16/11/28 08:07:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/11/28 08:07:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/11/28 08:07:19 INFO mapred.FileInputFormat: Total input paths to process : 1
16/11/28 08:07:19 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1281)
	at java.lang.Thread.join(Thread.java:1355)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:862)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:600)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:789)
16/11/28 08:07:19 INFO mapreduce.JobSubmitter: number of splits:2
16/11/28 08:07:19 INFO mapreduce.JobSubmitter: Submitting t

In [25]:
!hdfs dfs -get nbmodelNoSmooth/part-00000 /home/cloudera/Downloads/scalingML-master/SPAM_Model_MNB3.tsv
print "Bottom 10"
!head SPAM_Model_MNB3.tsv
print "Top 10"
!tail SPAM_Model_MNB3.tsv

Bottom 10
considered  	0.0 	0.000120627261761
screaming  	0.0 	0.000120627261761
sending  	0.0 	0.000120627261761
audio  	0.0 	0.000241254523522
linda's  	0.00013691128149 	0.0
wood  	0.00013691128149 	0.0
advice  	0.00013691128149 	0.000361881785283
writing  	0.00013691128149 	0.000723763570567
scotty  	0.00013691128149 	0.0
reclaimers  	0.0 	0.000120627261761
Top 10
my  	0.0052026286966 	0.00772014475271
shall  	0.00013691128149 	0.000965018094089
percent  	0.0 	0.000120627261761
scenario  	0.00013691128149 	0.0
limitless  	0.0 	0.000241254523522
click  	0.00013691128149 	0.00470446320869
abidjan  	0.0 	0.000241254523522
romeo  	0.00013691128149 	0.0
utilities'transmission  	0.000273822562979 	0.0
entirely  	0.0 	0.000241254523522


## 2.3.4

HW 2.3.4 Classify Documents using the learnt Multinomial Naive Bayes model using Hadoop Streaming

Classify each Enron email messages using the learnt Naive Bayes classifier (Testing on the training set is bad practice but we will allow that here to simplify the work here).



In [37]:
import io
import re
import math
from collections import defaultdict

f = open('/home/cloudera/Downloads/scalingML-master/SPAM_Model_MNB3.tsv', "r")

## use readlines to read all lines in the file
## The variable "lines" is a list containing all lines
lines = file.read(f).splitlines()
#print lines

modelSpam = {}
modelHam = {} #defaultdict(int)

for line in lines:
        #print line
        word, h, s = line.split('\t')
        word = word.strip(" ")
        modelHam[word] = float(h) *1.0
        modelSpam[word] = float(s) *1.0
f.close()

#get the lines from mapper
currentID = 0
vocab = set() # all the words in email
accurate = 0
total = 0
f = open('/home/cloudera/NaiveBayes/enronemail_1h.txt', "r")


## use readlines to read all lines in the file
## The variable "lines" is a list containing all lines
lines = file.read(f).splitlines()
#a="shanghai shangha 6 7"
p_h_z = 0 #number of zero probs
p_s_z = 0

#print a.split('\t')
for line in lines:
    try:
    #print line
        iD, spam, subject, body = line.split('\t')
        words = body + ' ' + subject
    except ValueError:
        iD, spam, words = line.split('\t')
    #iD, spam, words = line.split('\t')
    p_h = 1.0
    p_s = 1.0
    for word in re.findall(r"[\w']+", words):
        #words.split():
        word = word.lower()
        #print "words" , word, float(modelHam[word])
        if modelHam[word]>0.0 and p_h > 0.0:
            if p_h is not 1.0:
                #print p_h
                p_h  = math.log(p_h)+math.log(float(modelHam[word]))
               # print p_h
                p_h = math.exp(p_h)
                #print p_h
            else: #
                p_h = float(modelHam[word])
        else:
            p_h = float(modelHam[word])
        
        
        if modelSpam[word]>0.0 and p_s > 0.0:
            if p_s is not 1.0:
                p_s  = math.log(p_s)+math.log(float(modelSpam[word]))
                p_s = math.exp(p_s)
                #print p_s
            else:
                p_s = float(modelSpam[word])
        else:
            p_s = float(modelSpam[word])
        
    if(int(spam)== 1):
        c = "SPAM"
    else:
        c = "HAM"
        
    if p_s == 0.0 :
        p_s_z +=1
    if p_h == 0.0 :
        p_h_z +=1
    if(p_s != 0.0 or p_h != 0.0):
        if p_s > p_h:
            p_c = "SPAM"
        else:
            p_c = "HAM"
        print iD, c, p_c, p_h, p_s

        if(p_c == c):

            accurate += 1
        total +=1

    # vocab.add(word)
    #currentID = iD
        
print "Accuracy: ", (accurate*1.0/total)
print "Zeros Ham: ", p_h_z, " Spam: ",p_s_z
print "Accuracy for non-zero " , (accurate)*1.0/(total - (p_h_z+p_s_z))

0001.1999-12-10.farmer HAM SPAM 1.92422639962e-18 0.000844390832328
0001.1999-12-10.kaminski HAM HAM 4.04068301655e-12 0.0
0001.2000-01-17.beck HAM HAM 1.02162401469e-115 0.0
0001.2000-06-06.lokay HAM SPAM 1.77014802049e-132 0.0182147165259
0001.2001-02-07.kitchen HAM SPAM 8.46723575782e-133 2.10628751941e-11
0001.2001-04-02.williams HAM HAM 7.80699305596e-212 0.0
0002.1999-12-13.farmer HAM SPAM 2.54625763151e-162 3.05569661879e-07
0002.2001-02-07.kitchen HAM HAM 1.41538955021e-175 0.0
0002.2001-05-25.SA_and_HP SPAM SPAM 0.0 2.34010218775e-233
0002.2003-12-18.GP SPAM HAM 1.35433981219e-12 1.65363521005e-176
0002.2004-08-01.BG SPAM SPAM 0.0 1.44049779701e-319
0003.1999-12-10.kaminski HAM HAM 6.42997382469e-161 0.0
0003.1999-12-14.farmer HAM HAM 1.42335387812e-31 0.0
0003.2000-01-17.beck HAM HAM 2.14404458219e-166 0.0
0003.2001-02-08.kitchen HAM SPAM 6.69862024157e-196 2.10628751941e-11
0003.2003-12-18.GP SPAM SPAM 0.0 0.000120627261761
0003.2004-08-01.BG SPAM SPAM 0.0 3.96434843705e-304

#3.4

In [38]:
%%writefile reducer341.py
#!/usr/bin/env python
#Laplace smoothing

import sys
import re

from collections import defaultdict
spamWords = defaultdict(int)
hamWords = defaultdict(int)
spamCount = 0
hamCount = 0
vocab = set() 

p_word_spam = defaultdict(int)
p_word_ham = defaultdict(int)

for line in sys.stdin:
    iD, spam, word, count = line.split('\t')
    vocab.add(word)
    if(int(spam) == 1):
        spamWords[word] += int(count)
        spamCount += 1
    else:
        hamWords[word] += int(count)
        hamCount +=1


for word in vocab:
    p_word_spam[word] = (1+spamWords[word]*1.00)/spamCount*1.0
    p_word_ham[word] = (1+hamWords[word]*1.00)/hamCount*1.0

    print word, "\t", p_word_ham[word], "\t", p_word_spam[word] #, "\t", p_word_ham[word]


Overwriting reducer341.py


In [39]:
!chmod a+x /home/cloudera/Downloads/scalingML-master/reducer341.py

In [40]:
#### Model for enrollment
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -D mapreduce.partition.keycomparator.options="-k1,1" \
   -mapper /home/cloudera/Downloads/scalingML-master/mapper321.py \
   -reducer /home/cloudera/Downloads/scalingML-master/reducer341.py \
   -input NB/e3.txt\
   -output nbmodelLaplace99

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob188277713146083640.jar tmpDir=null
16/11/28 09:44:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/11/28 09:44:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/11/28 09:44:39 INFO mapred.FileInputFormat: Total input paths to process : 1
16/11/28 09:44:39 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1281)
	at java.lang.Thread.join(Thread.java:1355)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:862)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:600)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:789)
16/11/28 09:44:39 INFO mapreduce.JobSubmitter: number of splits:2
16/11/28 09:44:40 INFO mapreduce.JobSubmitter: Submitting to

In [41]:
!hdfs dfs -get nbmodelLaplace99/part-00000 /home/cloudera/Downloads/scalingML-master/LaplaceSPAM_Model_MNB3.tsv

In [42]:
import io
import re
import math
from collections import defaultdict

f = open('/home/cloudera/Downloads/scalingML-master/LaplaceSPAM_Model_MNB3.tsv', "r")

## use readlines to read all lines in the file
## The variable "lines" is a list containing all lines
lines = file.read(f).splitlines()
#print lines

modelSpam = {}
modelHam = {} #defaultdict(int)

for line in lines:
        #print line
        word, h, s = line.split('\t')
        word = word.strip(" ")
        modelHam[word] = float(h) *1.0
        modelSpam[word] = float(s) *1.0
f.close()

#get the lines from mapper
currentID = 0
vocab = set() # all the words in email
accurate = 0
total = 0
f = open('/home/cloudera/NaiveBayes/enronemail_1h.txt', "r")


## use readlines to read all lines in the file
## The variable "lines" is a list containing all lines
lines = file.read(f).splitlines()
#a="shanghai shangha 6 7"


#print a.split('\t')
for line in lines:
    try:
    #print line
        iD, spam, subject, body = line.split('\t')
        words = body + ' ' + subject
    except ValueError:
        iD, spam, words = line.split('\t')
    #iD, spam, words = line.split('\t')
    p_h = 1.0
    p_s = 1.0
    for word in re.findall(r"[\w']+", words):
        #words.split():
        word = word.lower()
        #print "words" , word, float(modelHam[word])
        if modelHam[word]>0.0 and p_h > 0.0:
            if p_h is not 1.0:
                #print p_h
                p_h  = math.log(p_h)+math.log(float(modelHam[word]))
               # print p_h
                p_h = math.exp(p_h)
                #print p_h
            else: #
                p_h = float(modelHam[word])
        else:
            p_h = float(modelHam[word])
        
        
        if modelSpam[word]>0.0 and p_s > 0.0:
            if p_s is not 1.0:
                p_s  = math.log(p_s)+math.log(float(modelSpam[word]))
                p_s = math.exp(p_s)
                #print p_s
            else:
                p_s = float(modelSpam[word])
        else:
            p_s = float(modelSpam[word])
        
    if(int(spam)== 1):
        c = "SPAM"
    else:
        c = "HAM"
        
    if p_s == 0.0:
        print "Zero"

    if p_s > p_h:
        p_c = "SPAM"
    else:
        p_c = "HAM"
    print iD, c, p_c, p_h, p_s

    if(p_c == c):

        accurate += 1
    total +=1

    # vocab.add(word)
    #currentID = iD
        
print "Accuracy: ", (accurate*1.0/total)

0001.1999-12-10.farmer HAM HAM 2.4245252635e-17 1.63458429239e-18
0001.1999-12-10.kaminski HAM HAM 1.02204684856e-11 2.82024022504e-12
0001.2000-01-17.beck HAM HAM 1.54404228035e-76 3.42464715543e-311
0001.2000-06-06.lokay HAM HAM 3.55816390931e-98 4.42873182711e-256
0001.2001-02-07.kitchen HAM HAM 5.38631368375e-130 2.99390192822e-153
0001.2001-04-02.williams HAM HAM 5.61501985991e-195 2.36439762862e-203
0002.1999-12-13.farmer HAM SPAM 1.09343729309e-132 4.6562996096e-07
0002.2001-02-07.kitchen HAM HAM 5.47054257876e-169 4.68931226745e-172
0002.2001-05-25.SA_and_HP SPAM SPAM 1.1531672766e-253 1.09024749429e-225
0002.2003-12-18.GP SPAM SPAM 6.31758887532e-217 2.50762805245e-158
0002.2004-08-01.BG SPAM HAM 2.10818244343e-15 6.96296340852e-312
0003.1999-12-10.kaminski HAM HAM 2.33817527134e-157 1.59453964537e-179
0003.1999-12-14.farmer HAM HAM 1.29922929426e-30 1.12197490023e-37
0003.2000-01-17.beck HAM HAM 7.07691707779e-156 1.12659077551e-253
0003.2001-02-08.kitchen HAM HAM 2.412165623

In [ ]:
Accuracy increases, because the probability of a message bei

3. HW2.6 Benchmark your code with the Python SciKit-Learn

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import feature_extraction.text 
import csv, re, string
import numpy as np

filename = open('enron-1.txt', 'r')
txt = filenameread().strip()

emails = txt.split('\n')
train_label = {}
train_data ={}
test_label ={}
test_data = {}

for email in emails:
    eid, tr_l, tr_data = email.strip().split('\t') 
    train_label.add(tr_l)
    test_label.add(tr_l)
    #get the data
